In [2]:
import random
from dataclasses import dataclass, field
from typing import List

In [465]:
# Poker Hands

# (10)* Royal Flush (AD, KD, QD, JD, TD)
# (9)* Straight Flush (8C, 7C, 6C, 5C, 4C)
# (8)* Four of a Kind (JH, JD, JS, JC, 7D)
# (7)* Full House (TH, TD, TS, 9C, 9D)
# (6)* Flush (4C, JC, 8C, 2C, 9C) - kicker
# (5)* Straight (9C, 8D, 7S, 6D, 5H) - kicker
# (4)* Three of a kind (7C, 7D, 7S, KC, 3D)
# (3)* Two Pair (4C, 4S, 3C, 4D, QC) - kicker
# (2)* Two of a kind (1D, 1H, 2S, 4C, 5D) - kicker
# (1)* High Card( 3D, JC, 8S, 4H, 2C) - kicker


In [3]:
RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = "H D C S".split()

def create_deck():
    return [Card(r, s) for s in SUITS for r in RANKS]

@dataclass
class Card:
    rank: str
    suit: str

    rmap = {'1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'T':10, 'J':11, 'Q':12, 'K':13, 'A':14}
    def val(self):
        return self.rmap[self.rank]

@dataclass
class Deck:
    cards: List[Card] = field(default_factory=create_deck)
    
    def deal(self):
        return self.cards[5]
    
@dataclass
class Hand:
    cards = []
    
    def __init__(self, hand):
        vals = hand.split()
        for c in vals:
            card = Card(c[0], c[1])
            self.cards.append(card)
    
    def ranks(self):
        vals = []
        for card in self.cards:
            vals.append(card.val())
        return sorted(vals, reverse=True)
        
# card = Card('5', "H")
# print(card)
# card = Card('K', 'D')
# print(card)
# deck = Deck()
# print(deck.deal())
hand = Hand('QS 4H AS 2D 8C')
print(hand.cards)
print(hand.ranks())

[Card(rank='Q', suit='S'), Card(rank='4', suit='H'), Card(rank='A', suit='S'), Card(rank='2', suit='D'), Card(rank='8', suit='C')]
[14, 12, 8, 4, 2]


In [ ]:
def flush(hand):
    # hand must have 5 cards
    if len(hand) != 5: return False 
    # extract the suit from each card,
    # adding suit to a set. If flush, 
    # set will contain one element
    suits = set()
    for card in hand:
        suits.add(card[1])
    if len(suits) == 1:
        return True, (sorted(card_ranks(hand), reverse=True))
    return False

def test_flush():
    hand = Hand()

In [563]:
rank_map = {'1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'T':10, 'J':11, 'Q':12, 'K':13, 'A':14}

def card_ranks(hand):
    cards = [card[0] for card in hand]
    rank_vals = [rank_map[card] for card in cards]
    return sorted(rank_vals, reverse=True)

def poker(hands):
    "Return the best hand: poker([hand,...]) => hand"
    return max(hands, key=hand_rank)


def hand_rank(hand):
    "Return a value indicating the ranking of a hand."
    ranks = card_ranks(hand)
    rank_vals = [rank_map[rank] for rank in ranks]
    print(straight(ranks)[0])
    print(flush(ranks)[0])
    if straight(ranks)[0] and flush(hand)[0]:
        if max(rank_vals) == 14:
            return (10,)
        else:
            return (9,)
    elif kind(4, ranks):
        return (8,)
    elif full_house(ranks):
        return (7,)
    elif flush(hand):
        return (6, 0)
    elif straight(ranks):
        return (5, 0)
    elif kind(3, ranks):
        return (4,)
    elif two_pair(ranks):
        return (3, 0)
    elif kind(2, ranks):
        return (2, 0)
    else:
        return (1, 0)

def flush(hand):
    # hand must have 5 cards
    if len(hand) != 5: return False 
    # extract the suit from each card,
    # adding suit to a set. If flush, 
    # set will contain one element
    suits = set()
    for card in hand:
        suits.add(card[1])
    if len(suits) == 1:
        return True, (sorted(card_ranks(hand), reverse=True))
    return False

def straight(ranks):
    # hand must have 5 cards
    if len(ranks) != 5: return False 
    if len(set(ranks)) != 5: return False
    # convert ranks to numerical values
    rank_vals = []
    for rank in ranks:
        rank_vals.append(rank_map[rank])
    if max(rank_vals)-min(rank_vals)+1 == 5:
        return True, sorted(rank_vals, reverse=True)
    return False

# 1 1 1 2 2 (2)
def full_house(ranks):
    if len(ranks) != 5: return False
    if len(set(ranks)) != 2: return False
    if ranks.count(ranks[0]) == 3 or ranks.count(ranks[0]) == 2: return True
    return False

# 1 1 2 2 3 (3) 2 2 2 2 1
# 1 1 2 4 5 (4) 2 2 1 1 1
# 44332  (3) 2 2 2 2 1
def two_pair(ranks):
    if len(ranks) != 5: return False
    counts = [ranks.count(ranks[x]) for x in range(5)]
    if max(counts) != 2 or min(counts) != 1: return False
    if "".join(str(counts)).count("2") != 4: return False
    return True
    

def kind(num, ranks):
    # 1 1 1 1 2 (2) 4 4 4 4 1
    # 1 1 1 2 2 (2) 3 3 3 2 2
    # 1 1 1 2 3 (3) 3 3 3 1 1
    # 1 1 2 2 3 (3) 2 2 2 2 1
    # 1 1 2 3 4 (4) 2 1 1 1 1
    # 1 2 3 4 5 (5) 1 1 1 1 1
    if len(ranks) != 5: return False
    counts = [ranks.count(ranks[x]) for x in range(5)]
    uniq = len(set(ranks))
    # four of a kind
    if num == 4:
        if max(counts) != 4: 
            return False
        return True
    if num == 3:  
        if max(counts) != 3 or min(counts) != 1: return False
        return True
    if num == 2:
        if uniq != 4: return False
        return True
    return False

    
# 1 2 3 4 5 (5) 1 1 1 1 1
# T 8 6 4 2 -> 10 8 6 4 2
# T 8 7 5 3 0> 10 8 7 5 1
# 0 0 -1 -1 1
def high(ranks):
    if len(ranks) != 5: return 0
    rank_vals = [rank_map[rank] for rank in ranks]
    sorted_ranks = []
    sorted_ranks == rank_vals.sort()
    #val = sorted_ranks
    #num = []''.join(v) for v in val]
    print(sorted_ranks)
    return max(rank_vals)

    


#### Test Methods

In [579]:
# (9)* Royal Flush (AD, KD, QD, JD, TD)
# (8)* Straight Flush (8C, 7C, 6C, 5C, 4C)
# (7)* Four of a Kind (JH, JD, JS, JC, 7D)
# (6)* Full House (TH, TD, TS, 9C, 9D)
# (5)* Flush (4C, JC, 8C, 2C, 9C) (Kicker needed)
# (4)* Straight (9C, 8D, 7S, 6D, 5H) (Kicker needed)
# (3)* Three of a kind (7C, 7D, 7S, KC, 3D)
# (2)* Two Pair (4C, 4S, 3C, 3D, QC) (Kicker needed)
# (1)* Two of a kind (1D, 1H, 2S, 4C, 5D) (Kicker needed)
# (0)* High Card( 3D, JC, 8S, 4H, 2C) (Kicker needed)

def test_poker():
    "Test cases for the functions in poker program."
    rf = "AC KC QC JC TC".split()
    sf = "6C 7C 8C 9C TC".split()
    fk = "9D 9H 9S 9C TD".split()
    fh = "TD TC TH 7C 7D".split()
    fl = "4C JC 8C 2C 9C".split()
    st = "9C 8D 7S 6D 5H".split()
    tk = "7C 7D 7S KC 3D".split()
    tp = "4C 4S 3C 3D QC".split()
    op = "1D 1H 2S 4C 5D".split()
    hc = "3D JC 8S 4H 2C".split()
    hc1 = "3D Q6 9S 4H 2C".split()
    hc2 = "8D 8C 7D 5S 4H".split()
    hc3 = "8D 8C 7D 5S 4H".split()
    hc4 = "8D 8C 6D 5S 4H".split()
    assert poker([rf, fl, tp, tk, hc, fh, sf]) == rf
    assert poker([sf, fk, fh]) == sf
    assert poker([fk, fh]) == fk
    assert poker([fh, st, op]) == fh
    assert poker([tk, st, fl]) == fl
    assert poker([hc, tp, st, tk]) == st
    assert poker([op, hc, tk, tp]) == tk
    assert poker([op, tp, hc]) == tp
    assert poker([op, hc, tk]) == tk
    assert poker([hc, hc, hc]) == hc
    assert poker([hc, hc1]) == hc1
    assert poker([fh]) == fh
    assert poker([sf] + 99*[fh]) == sf
    assert poker([hc2, hc3]) == hc2
    assert poker([hc4, hc3]) == hc3
    return "test_poker passes"

def test_royal_flush():
    hand = "TC JC QC KC AC"
    print(card_ranks(hand.split()))
    print(flush(hand.split())[0])
    print(straight(card_ranks(hand.split())))
    print(max(card_ranks(hand.split())))
#     rank_vals = [rank_map[rank] for rank in card_ranks(hand.split())]
#     assert flush(hand.split())[0] == True and straight(card_ranks(hand.split()))[0] == True and max(card_ranks(hand.split())) == 14
#     hand = "9C TC JC QC KC"
#     rank_vals = [rank_map[rank] for rank in card_ranks(hand.split())]
#     assert flush(hand.split())[0] == True and straight(card_ranks(hand.split()))[0] == True and max(rank_vals) == 13
    return "test_royal_flush passes"

def test_straight_flush():
    hand = "9C TC JC QC KC"
    rank_vals = [rank_map[rank] for rank in card_ranks(hand.split())]
    assert flush(hand.split()) == True and straight(card_ranks(hand.split())) == True and max(rank_vals) != 14
    return "test_royal_flush passes"

def test_straight_flush():
    rank_vals = [rank_map[rank] for rank in card_ranks("TC JC QC KC AC".split())]
    assert flush("TC JC QC KC AC".split()) == True and straight(card_ranks("TC JC QC KC AC".split())) == True and max(rank_vals) == 14
    assert flush("TC JC QC KC AC".split()) == True and straight(card_ranks("TC JC 3C KC AC".split())) == False and max(rank_vals) == 14
    assert flush("TC JS QC KC AC".split()) == False and straight(card_ranks("TC JC QC KC AC".split())) == True and max(rank_vals) == 14
    assert flush("TC JD QC KC AC".split()) == False and straight(card_ranks("TC JC QC KC 2C".split())) == False and max(rank_vals) == 14
    return "test_royal_flush passes"

def test_flush():
    assert flush( "6C 7C 8C 9C TC".split()) == (True, [10, 9, 8, 7, 6])
    assert flush( "6C 7C 8C 9C TC QC".split()) == False
    assert flush("9D 9H 9S 9C TD".split()) == False
    assert flush("3S 5S".split()) == False
    assert flush("".split()) == False
    return "test_flush passes"

def test_full_house():
    assert full_house(card_ranks("5C 5D 5S 1S 1H".split())) == True
    assert full_house(card_ranks("5C 5D 3S 1S 1H".split())) == False
    return "test_full_house passes"

def test_straight():
    assert straight("A K Q J T".split()) == (True, [14, 13, 12, 11, 10])
    assert straight("A Q K J T".split()) == (True, [14, 13, 12, 11, 10])
    assert straight("8 7 6 5 3".split()) == False
    assert straight("J J J J 7".split()) == False
    assert straight("T T T 9 9".split()) == False
    assert straight("4 J 8 2 9".split()) == False
    assert straight("9 8 7 6 5".split()) == (True, [9, 8, 7, 6, 5])
    assert straight("7 7 7 K 3".split()) == False
    assert straight("4 4 3 3 Q".split()) == False
    assert straight("1 1 2 4 5".split()) == False
    assert straight("9 6 7 8 T".split()) == (True, [10, 9, 8, 7, 6])
    assert straight("T K A J Q".split()) == (True, [14, 13, 12, 11, 10])
    assert straight("9 T J Q K".split()) == (True, [13, 12, 11, 10, 9])
    assert straight("8 T J Q K".split()) == False
    assert straight("9 T 1 Q K".split()) == False
    assert straight("9 9 9 9 T".split()) == False
    assert straight("3 5".split()) == False
    assert straight("6 7 8 9 T C".split()) == False
    assert straight("".split()) == False
    return "test_straight passes"

def test_four_of_a_kind():
    assert kind(4, "A K Q J T".split()) == False
    assert kind(4, "8 7 6 5 3".split()) == False
    assert kind(4, "J J J J 7".split()) == True
    assert kind(4, "T T T 9 9".split()) == False
    assert kind(4, "4 J 8 2 9".split()) == False
    assert kind(4, "9 8 7 6 5".split()) == False
    assert kind(4, "7 7 7 K 3".split()) == False
    assert kind(4, "4 4 3 3 Q".split()) == False
    assert kind(4, "1 1 2 4 5".split()) == False
    assert kind(4, "9 6 7 8 T".split()) == False
    assert kind(4, "T K A J Q".split()) == False
    assert kind(4, "9 T J Q K".split()) == False
    assert kind(4, "8 T J Q K".split()) == False
    assert kind(4, "9 T 1 Q K".split()) == False
    assert kind(4, "9 9 9 9 T".split()) == True
    assert kind(4, "A K Q J T".split()) == False
    assert kind(4, "8 7 6 5 3".split()) == False
    assert kind(4, "J J J J 7".split()) == True
    assert kind(4, "T T T 9 9".split()) == False
    assert kind(4, "4 J 8 2 9".split()) == False
    assert kind(4, "9 8 7 6 5".split()) == False
    assert kind(4, "7 7 7 K 3".split()) == False
    assert kind(4, "4 4 3 3 Q".split()) == False
    assert kind(4, "1 1 2 4 5".split()) == False
    assert high("3 5".split()) == 0
    assert high("6 7 8 9 T C".split()) == 0
    assert high("".split()) == 0
    return "test_four_of_a_kind passes"
    
def test_three_of_a_kind():
    assert kind(3, "A K Q J T".split()) == False
    assert kind(3, "8 7 6 5 3".split()) == False
    assert kind(3, "J J J J 7".split()) == False
    assert kind(3, "T T T 9 9".split()) == False
    assert kind(3, "4 J 8 2 9".split()) == False
    assert kind(3, "9 8 7 6 5".split()) == False
    assert kind(3, "7 7 7 K 3".split()) == True
    assert kind(3, "4 4 3 3 Q".split()) == False
    assert kind(3, "1 1 2 4 5".split()) == False
    return "test_three_of_a_kind passes"


def test_two_of_a_kind():
    assert kind(2, "A K Q J T".split()) == False
    assert kind(2, "8 7 6 5 3".split()) == False
    assert kind(2, "J J J J 7".split()) == False
    assert kind(2, "T T T 9 9".split()) == False
    assert kind(2, "4 J 8 2 9".split()) == False
    assert kind(2, "9 8 7 6 5".split()) == False
    assert kind(2, "7 7 7 K 3".split()) == False
    assert kind(2, "4 4 3 3 Q".split()) == False
    assert kind(2, "7 7 2 4 5".split()) == True
    assert kind(2, "7 7 6 2 4".split()) == True
    return "test_two_of_a_kind passes"

def test_two_pair():
    assert two_pair("A K Q J T".split()) == False
    assert two_pair("8 7 6 5 3".split()) == False
    assert two_pair("J J J J 7".split()) == False
    assert two_pair("T T T 9 9".split()) == False
    assert two_pair("4 J 8 2 9".split()) == False
    assert two_pair("9 8 7 6 5".split()) == False
    assert two_pair("7 7 7 K 3".split()) == False
    assert two_pair("4 4 3 3 Q".split()) == True
    assert two_pair("1 1 2 4 5".split()) == False
    assert two_pair("9 6 7 8 T".split()) == False
    assert two_pair("T K A J Q".split()) == False
    assert two_pair("9 T J Q K".split()) == False
    assert two_pair("8 T J Q K".split()) == False
    assert two_pair("9 T 1 Q K".split()) == False
    assert two_pair("9 9 9 9 T".split()) == False
    assert two_pair("3 5".split()) == False
    assert two_pair("6 7 8 9 T C".split()) == False
    assert two_pair("".split()) == False
    return "test_two_pair passes"


def test_high():
    assert high(card_ranks("9D TC 1S AH KD".split())) == 14
    assert high("A K Q J T".split()) == 14
    assert high("8 7 6 5 3".split()) == 8
    assert high("J J J J 7".split()) == 11
    assert high("T T T 9 9".split()) == 10
    assert high("4 J 8 2 9".split()) == 11
    assert high("9 8 7 6 5".split()) == 9
    assert high("7 7 7 K 3".split()) == 13
    assert high("4 4 3 3 Q".split()) == 12
    assert high("1 1 2 4 5".split()) == 5
    assert high("9 6 7 8 T".split()) == 10
    assert high("T K A J Q".split()) == 14
    assert high("9 T J Q K".split()) == 13
    assert high("8 T J Q K".split()) == 13
    assert high("9 T 1 Q K".split()) == 13
    assert high("9 9 9 9 T".split()) == 10
    assert high("9 9 9 9 J".split()) == 11
    assert high("3 5".split()) == 0
    assert high("6 7 8 9 T C".split()) == 0
    assert high("".split()) == 0
    return "test_high passes"

def test_hand_rank():
    assert hand_rank("AD KD QD JD TD".split()) == 100
    assert hand_rank("8C, 7C, 6C, 5C, 4C".split()) == 90
    assert hand_rank("JH, JD, JS, JC, 7D".split()) == 80
    assert hand_rank("TH, TD, TS, 9C, 9D)".split()) == 70
    assert hand_rank("4C, JC, 8C, 2C, 9C".split()) == 60
    assert hand_rank("9C, 8D, 7S, 6D, 5H".split()) == 50
    assert hand_rank("7C, 7D, 7S, KC, 3D".split()) == 40
    assert hand_rank("4C, 4S, 3C, 3D, QC".split()) == 30
    assert hand_rank("1D, 1H, 2S, 4C, 5D".split()) == 20
    assert hand_rank("9D TC 1S AH KD".split()) == 14
    assert hand_rank("9D TC 1S 5H KD".split()) == 13
    assert hand_rank("1D 2C 3S QH 5D".split()) == 12
    assert hand_rank("1D 2C 3S JH 5D".split()) == 11
    assert hand_rank("1D 2C TS 4H 5D".split()) == 10
    assert hand_rank("1D 2C 9S 4H 5D".split()) == 9
    assert hand_rank("1D 8C 3S 4H 5D".split()) == 8
    assert hand_rank("1D 7C 3S 4H 5D".split()) == 7
    assert hand_rank("2D 4C 1S 6H 5D".split()) == 6
    assert hand_rank("1D 2C 3S 4H 5D".split()) != 5
    assert hand_rank("1D 2C 3S 4H 1D".split()) != 4
    return "test_hand_rank passes"

def test_card_ranks():
    assert card_ranks("4D 7H 3C QS JC".split()) == [12, 11, 7, 4, 3]
    assert card_ranks("4C JC 8C 2C 9C".split()) == [11, 9, 8, 4, 2]
    assert card_ranks("2C JC 8C 4C 2D".split()) == [11, 8, 4, 2, 2]
    print("test_card_ranks passes")

def test_compare():
    assert (4, 3) > (4, 2, 1)
    assert (4, 2, 1) < (4, 3)
    assert (4, 3, 2) < (4, 5, 1)
    assert (13, 10, 2) > (13, 10, 1)
    assert (1, 2, 3) > (1, 2, 2)
    assert (1, 2, 3) == (1, 2, 3)
    assert (10,) > (5, 2, 1)
    print("test_compare passes")

In [580]:
print(test_royal_flush())
# print(test_straight_flush())
# print(test_four_of_a_kind())
# print(test_full_house())
print(test_flush())
print(test_straight())
# print(test_three_of_a_kind())
# print(test_two_pair())
# print(test_two_of_a_kind())
# print(test_high())
# print(test_hand_rank())
# print(test_poker())
test_card_ranks()
test_compare()

[14, 13, 12, 11, 10]
True


KeyError: 14